In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

In [ ]:
from automl.external_support.sb3.sb3_model_wrapper import SB3WrapperTorch

loaded_model = SB3WrapperTorch({"sb3_model" : "dqn-MountainCar-v0", "device" : "cpu"})

In [ ]:
model_name = "sb3_mountaincar_dqn"

## Perturb models randomly by percentage

In [ ]:
max_percentage = 1000000000000000000000000000000000000
min_percentage = 10000000000000

In [ ]:
from automl.ml.models.torch_model_components import perturb_model_parameters


perturb_model_parameters(loaded_model, min_percentage / 100.0, max_percentage / 100.0)

print(loaded_model.values)

In [ ]:

model_name = f"{model_name}_perturbed_{min_percentage}_{max_percentage}"



## Perturb model with gaussian noise

In [ ]:
avg = 0
std = 0.1
fraction = 0.2

In [ ]:
from automl.ml.models.torch_model_components import perturb_model_parameters_gaussian

perturb_model_parameters_gaussian(loaded_model, avg, std, fraction)


In [ ]:
model_name = f"{model_name}_gaussian_{avg}_{std}_{fraction}"


## Save Model

In [ ]:
model_store_path = "data\\models"


model_file_name = f"{model_name}.pkl"

In [ ]:
import pickle
import os

from automl.utils.files_utils import new_path_if_exists

os.makedirs(model_store_path, exist_ok=True)
model_path = new_path_if_exists(model_file_name, dir=model_store_path)


with open(model_path, "wb") as f:
    pickle.dump(loaded_model, f)